In this notebook we look for interactions between the odor and the v_fwd representations in G4S.

In [ ]:
%matplotlib inline
from copy import deepcopy
from itertools import product as cproduct
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression, Ridge

from aux import Generic
from data import DataLoader, random_chunk_split
from db import make_session, d_models
from lin_fit import fit_h_train_test, calc_r2
from plot import set_font_size

import CONFIG as C

Parameters

In [ ]:
TRAIN_LEN = 32  # s
TEST_LEN = 8  # s

EXPTS = ['white_odor_fluct']
TARG = 'G4S'

WDW = (-3, 1)

METHODS = {
    'odor_binary': 'built-in',
    'v_fwd': 'built-in',
}

PARAMS = {
    'odor_binary': {'model': LinearRegression},
    'v_fwd': {'model': LinearRegression},
}

Resuable subroutines

Linear models

In [ ]:
class ModelO(object):
    
    def __init__(self, trial):
        
        # run odor_binary --> TARG fit
        rslt = fit_h_train_test(
            trial=trial,
            x_name='odor_binary',
            y_name=TARG,
            wdw=WDW,
            train_len=TRAIN_LEN,
            test_len=TEST_LEN,
            method=METHODS['odor_binary'],
            params=PARAMS['odor_binary'],
            C=C)
        
        # store results
        for k, v in rslt.items():
            self.__dict__[k] = v

In [ ]:
class ModelVF(object):
    
    def __init__(self, trial):
        
        # run v_fwd --> TARG fit
        rslt = fit_h_train_test(
            trial=trial,
            x_name='v_fwd',
            y_name=TARG,
            wdw=WDW,
            train_len=TRAIN_LEN,
            test_len=TEST_LEN,
            method=METHODS['v_fwd'],
            params=PARAMS['v_fwd'],
            C=C)
        
        # store results
        for k, v in rslt.items():
            self.__dict__[k] = v

In [ ]:
class ModelOVFI(object):
    
    def __init__(self, trial):
        
        # run odor_binary --> TARG fit
        self.o = fit_h_train_test(
            trial=trial,
            x_name='odor_binary',
            y_name=TARG,
            wdw=WDW,
            train_len=TRAIN_LEN,
            test_len=TEST_LEN,
            method=METHODS['odor_binary'],
            params=PARAMS['odor_binary'],
            C=C)
        
        # run v_fwd --> TARG fit
        self.vf = fit_h_train_test(
            trial=trial,
            x_name='v_fwd',
            y_name=TARG,
            wdw=WDW,
            train_len=TRAIN_LEN,
            test_len=TEST_LEN,
            method=METHODS['v_fwd'],
            params=PARAMS['v_fwd'],
            C=C)
        
        self.trial_name = trial.name
        self.y_name = TARG
        
        self.train = self.o.train & self.vf.train
        self.test = self.o.test & self.vf.test
        self.t = self.o.t
        
        self.y = self.o.y
        
        self.y_hat_train = self.o.y_hat_train + self.vf.y_hat_train
        self.y_hat_test = self.o.y_hat_test + self.vf.y_hat_test
        
        self.r2_train = calc_r2(self.y, self.y_hat_train)
        self.r2_test = calc_r2(self.y, self.y_hat_test)

In [ ]:
class ModelOVFJ(object):
    
    def __init__(self, trial):
        
        # run joint (odor_binary, v_fwd) --> TARG fit
        rslt = fit_hs_train_test(
            trial=trial,
            x_names=('odor_binary', 'v_fwd'),
            y_name=TARG,
            wdws={'odor_binary': WDW, 'v_fwd': WDW},
            train_len=TRAIN_LEN,
            test_len=TEST_LEN,
            methods=METHODS,
            params=PARAMS,
            C=C)
        
        self.trial_name = trial.name
        self.y_name = TARG
        
        self.train = rslt.train
        self.test = rslt.test
        self.t = rslt.t
        
        self.xs = rslt.xs
        self.y = rslt.y
        
        self.y_hat_train = rslt.y_hat_train
        self.y_hat_test = rslt.y_hat_test
        
        self.r2_train = calc_r2(self.y, self.y_hat_train)
        self.r2_test = calc_r2(self.y, self.y_hat_test)

### Plot true vs predicted G4S for each trial/model